In [75]:
import torch
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
import hashlib


In [76]:
# Correctly formatted URL for a local Elasticsearch instance
es = Elasticsearch(["http://localhost:9200"])

In [77]:
# Check the health of the cluster
health = es.cluster.health()
print(health)

{'cluster_name': 'elasticsearch', 'status': 'green', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 0, 'active_shards': 0, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 0, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 100.0}


In [79]:
# Ensure the entire text in each cell is displayed without truncation
pd.set_option('display.max_colwidth', None)

# Adjusting display width might not be necessary depending on your pandas version, but it's here for completeness
pd.set_option('display.width', None)

# Load your DataFrame
pew = pd.read_csv('../dataset/pew_dataset/metadata.csv')
columns = ['title','caption']

# Filtering the DataFrame to include only the specified columns
pew_df = pew[columns]

In [80]:
pew_df.head()

,title,caption
0,"Foreign-born population in the United States, 1850-2018","The foreign-born population residing in the U.S. reached a record 44.8 million, or 13.7% of the U.S. population, in 2018. This immigrant population has more than quadrupled since the 1960s, when the 1965 Immigration and Naturalization Act took effect. Though growth has begun to slow in recent years, the number of immigrants living in the United States is projected to almost double by 2065."
1,"English proficiency among U.S. immigrants, 1980-2018","Since 1980, the share of immigrants who are proficient in English (those who speak only English at home or speak English at least “very well”) has declined, though it has increased slightly in recent years. This decline has been driven entirely by those who speak only English at home, which fell from 30% of immigrants ages 5 and older in 1980 to 17% in 2018. The share who speaks English “very well,” meanwhile, has increased slightly, from 27% to 37% over the same time period."
2,"Languages spoken among U.S. immigrants, 2018","Among the nation’s immigrants, Spanish is by far the most spoken non-English language (42% of immigrants say they speak Spanish at home), but it is not the only non-English language spoken by immigrants. Some 6% of immigrants speak Chinese (including Mandarin and Cantonese), 5% speak Hindi or a related language, 4% speak Filipino or Tagalog, 3% speak Vietnamese, 3% speak French and 2% speak Dravidian."
3,"Hispanic population in the U.S., 2000-2017","There were nearly 60 million Latinos in the United States in 2017, accounting for approximately 18% of the total U.S. population. In 1980, with a population of 14.8 million, Hispanics made up just 6.5% of the total U.S. population. For more, read the accompanying blog post, “Key facts about U.S. Hispanics and their diverse heritage.” For facts on the foreign-born population in the United States, see our profile on U.S. immigrants."
4,Weekly broadcast audience for top 20 NPR-affiliated radio stations,"The top 20 NPR-affiliated public radio stations (by listenership) had on average a total weekly listenership of about 11 million in 2018, about the same as in 2017. (This includes listeners of NPR programming as well as original or other syndicated content aired on these stations.)"


In [81]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [82]:
index_name = "documents"

In [83]:
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body={
        "mappings": {
            "properties": {
                "title": {"type": "text"},
                "content": {"type": "text"},
                "embedding": {"type": "dense_vector", "dims": 384}
            }
        }
    })

In [84]:
# Check if the index exists
exists = es.indices.exists(index=index_name)
print(f"Index Exists: {exists}")

# Get index information (settings and mappings)
if exists:
    index_info = es.indices.get(index=index_name)
    print(index_info)

Index Exists: True
{'documents': {'aliases': {}, 'mappings': {'properties': {'content': {'type': 'text'}, 'embedding': {'type': 'dense_vector', 'dims': 384, 'index': True, 'similarity': 'cosine'}, 'title': {'type': 'text'}}}, 'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'number_of_shards': '1', 'provided_name': 'documents', 'creation_date': '1709672865336', 'number_of_replicas': '1', 'uuid': '4U_yVTmYTziNKUy-6RPndg', 'version': {'created': '8500010'}}}}}


In [87]:
for _, row in pew_df.iterrows():
    # Concatenate title and caption with a space or some delimiter
    combined_text = row['title'] + ". " + row['caption']
    
    # Generate embedding for the combined text
    embedding = model.encode(combined_text).tolist()

    # Generate a unique ID for the document using a hash of the title and caption
    unique_id = hashlib.sha256(combined_text.encode('utf-8')).hexdigest()
    
    # Index the document with the combined embedding and use the unique_id as the document ID
    es.index(index=index_name, id=unique_id, body={
        "title": row['title'],
        "content": row['caption'],
        "embedding": embedding
    })

In [88]:
doc_count = es.count(index=index_name)['count']
print(f"Number of documents in the index: {doc_count}")

Number of documents in the index: 1480
